In [1]:
linesCodeAnalyzer = None
with open('CodeJamCrawler/dataset_csvs/code_analyzer_out_sorted.csv', 'r') as f:
    linesCodeAnalyzer = f.readlines()
    
codeAnalyzerHeader = linesCodeAnalyzer[0]
linesCodeAnalyzer = linesCodeAnalyzer[1:]

linesCccc = None
with open('CodeJamCrawler/dataset_csvs/cccc_cleared.csv', 'r') as f:
    linesCccc = f.readlines()
    
linesASTExtractor = None
with open('ASTExtractor/ast_features.csv', 'r') as f:
    linesASTExtractor = f.readlines()

ASTExtractorHeader = linesASTExtractor[0]
    
ccccHeader = linesCccc[0]
linesCccc = linesCccc[1:]
linesASTExtractor = linesASTExtractor[1:]

print (linesCodeAnalyzer[0])
print (linesCccc[0])
print (ASTExtractorHeader)

0.0_14_16505_55.cpp,1,49,13,47,0,2,23.50,0.00,23.50,0.96,47,0,2

0.0_14_16505_55.csv;1;43;43.000;12;12.000;0;0.000;******;******;0;0.000;0;0.000;0;0.000;3

#file_name;cnt_classes;max_member_funs;max_nested_loops;max_nesting_depth;max_params_in_decl;member_funs;member_vars;min_member_funs



In [2]:
import os
def get_file_name(line, splitter):
    entries = line.split(splitter)
    file_name, file_ext = os.path.splitext(entries[0])
    return file_name

In [3]:
print (get_file_name(linesCodeAnalyzer[0], ','))
print (get_file_name(linesCccc[0], ';'))
print (get_file_name(linesASTExtractor[0], ';'))

0.0_14_16505_55
0.0_14_16505_55
0.0_14_16505_55


In [4]:
# removing all the entries that don't exist in outputCleared
cpp_files_only = []
for i in range(len(linesCodeAnalyzer)):
    if i % 1000 == 0:
        print (i)
        
    other = linesCodeAnalyzer[i]
    file_name_other = get_file_name(other, ',')
    
    found = False
    for cleared in linesCccc:
        file_name_cleared = get_file_name(cleared, ';')
        if file_name_cleared == file_name_other:
            found = True
            break
    if not found:
        continue
    cpp_files_only.append(file_name_other)
    
print ("Number of cpp files:", len(cpp_files_only))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
Number of cpp files: 18667


In [5]:
print (cpp_files_only[0])

0.0_14_16505_55


In [6]:
print ('Code analyzer output:', codeAnalyzerHeader)
print ('CCCC analyzer output:', ccccHeader)
print ('ASTExtractor output:', ASTExtractorHeader)

Code analyzer output: #File Name,Files,Lines,AVG Len,Code,Comments,White SP,Cd/Cm+WS,Cd/Cm,Cd/WS,% Code,Cd/File,Cm/File,WS/File,

CCCC analyzer output: #filename;number_of_modules;lines_of_code;lines_of_code_per_module;McCabes_cyclomatic_complexity;McCabes_cyclomatic_complexity_per_module;lines_of_comment;lines_of_comment_per_module;lines_of_code_per_line_of_comment;McCabes_cyclomatic_complexity_per_line_of_comment;IF4;IF4_per_module;IF4_visible;IF4_visible_per_module;IF4_concrete;IF4_concrete;rejected_lines_of_code

ASTExtractor output: #file_name;cnt_classes;max_member_funs;max_nested_loops;max_nesting_depth;max_params_in_decl;member_funs;member_vars;min_member_funs



In [7]:
import numpy as np

def parse_cccc(cccc_line):
    name = get_file_name(cccc_line, ';')
    res = name.split('_')[-1]
    author = name
    features = []
    for num in cccc_line.split(';')[1:]:
        if num == '******' or num == '------':
            num = '0.0'
        try:
            features.append(float(num))
        except ValueError:
            print (num)
            break
    return author, features, res


In [8]:
def parse_code_analyzer(code_anal_line):
    name = get_file_name(code_anal_line, ',')
    res = name.split('_')[-1]
    author = name
    features = []
    for num in code_anal_line.split(',')[1:]:
        try:
            features.append(float(num))
        except ValueError:
            print (num)
            break
    return author, features, res

In [9]:
def parse_ast_extractor(code_extractor_line):
    name = get_file_name(code_extractor_line, ';')
    res = name.split('_')[-1]
    author = name#.split('_')[0]
    features = []
    for num in code_extractor_line.split(';')[1:]:
        try:
            features.append(float(num))
        except ValueError:
            print (num)
            break
    return author, features, res

In [10]:
print ('cccc line vec', parse_cccc(linesCccc[0]))
print ('code line vec', parse_code_analyzer(linesCodeAnalyzer[0]))
print ('ast line vec', parse_ast_extractor(linesASTExtractor[0]))

cccc line vec ('0.0_14_16505_55', [1.0, 43.0, 43.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0], '55')
code line vec ('0.0_14_16505_55', [1.0, 49.0, 13.0, 47.0, 0.0, 2.0, 23.5, 0.0, 23.5, 0.96, 47.0, 0.0, 2.0], '55')
ast line vec ('0.0_14_16505_55', [0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 150000.0], '55')


In [11]:
cpp_files_map = set()

for n in cpp_files_only:
    cpp_files_map.add(n)

src_file_vec_map = {}

for cccc_line in linesCccc:
    name, features, res = parse_cccc(cccc_line)
    if name not in src_file_vec_map and name in cpp_files_map:
        src_file_vec_map[name] = [(features,res)]
    elif name in cpp_files_map:
        src_file_vec_map[name].append((features, res))

for code_anal_line in linesCodeAnalyzer:
    name, features, res = parse_code_analyzer(code_anal_line)
    if name in cpp_files_map:
        src_file_vec_map[name].append((features, res))

i = 0
for ast_line in linesASTExtractor:
    name, features, res = parse_ast_extractor(ast_line)
    if name in cpp_files_map:
        src_file_vec_map[name].append((features, res))
        i += 1
        
print (i, len(cpp_files_only), len(cpp_files_map))

18667 18667 18667


In [12]:
def get_author_name(file_name):
    return file_name.split('_')[0]

In [13]:
src_vec_map = {}

for key in src_file_vec_map.keys():
    v1 = src_file_vec_map[key][0]
    v2 = src_file_vec_map[key][1]
    v3 = src_file_vec_map[key][2]
    author = get_author_name(key)
    
    if author not in src_vec_map.keys():
        src_vec_map[author] = [(v1,v2,v3)]
    else:
        src_vec_map[author].append((v1, v2, v3))

In [14]:
print (len(src_vec_map))

sample_count = 0
for k in src_vec_map:
    sample_count += len(src_vec_map[k])
    
print ("Number of samples: ", sample_count)

1822
Number of samples:  18667


In [15]:
import random

def load_dataset(src_vec_map):
    dataset_list = []
    
    for k in src_vec_map:
        for i in range(len(src_vec_map[k])):
            label = src_vec_map[k][i][0][1]
            l1 = np.asarray(src_vec_map[k][i][0][0])
            l2 = np.asarray(src_vec_map[k][i][1][0])
            l3 = np.asarray(src_vec_map[k][i][2][0])
            l = np.concatenate((l1, l2), axis=0)
            l = np.concatenate((l, l3), axis=0)
            dataset_list.append((l, (k, label)))
    dataset = np.array(dataset_list)
    dataset = dataset.reshape((dataset.shape[0], dataset.shape[1]))
    return dataset

In [16]:
dataset = load_dataset(src_vec_map)
print (dataset.shape)
print (dataset[0])

(18667, 2)
[ array([  1.00000000e+00,   5.80000000e+01,   5.80000000e+01,
         3.20000000e+01,   3.20000000e+01,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         2.00000000e+00,   1.00000000e+00,   9.00000000e+01,
         1.70000000e+01,   7.40000000e+01,   0.00000000e+00,
         1.60000000e+01,   4.62000000e+00,   0.00000000e+00,
         4.62000000e+00,   8.20000000e-01,   7.40000000e+01,
         0.00000000e+00,   1.60000000e+01,   0.00000000e+00,
         0.00000000e+00,   1.00000000e+00,   2.00000000e+00,
         2.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.50000000e+05])
 ('mrsud', '60')]


In [17]:
def get_author_features_and_scores(author_name, dataset):
    codes = []
    scores = []
    
    for item in dataset:
        if item[1][0] == author_name:
            codes.append(item[0])
            scores.append(item[1][1])
            
    return (codes, scores)  

In [18]:
author_names = list(src_vec_map.keys())
X_list = []

for author in author_names:
    codes, scores = get_author_features_and_scores(author, dataset)
    X_list.extend(codes)
    
X = np.asarray(X_list)

In [19]:
print (codeAnalyzerHeader.split(',')[1:-1], "\n")
print (ccccHeader.split(';')[1:], "\n")
print (ASTExtractorHeader.split(';')[1:-1])
labels = ccccHeader.split(';')[1:] + codeAnalyzerHeader.split(',')[1:-1] + ASTExtractorHeader.split(';')[1:]

print ("\nTotal label length before clearing:", len(labels))

['Files', 'Lines', 'AVG Len', 'Code', 'Comments', 'White SP', 'Cd/Cm+WS', 'Cd/Cm', 'Cd/WS', '% Code', 'Cd/File', 'Cm/File', 'WS/File'] 

['number_of_modules', 'lines_of_code', 'lines_of_code_per_module', 'McCabes_cyclomatic_complexity', 'McCabes_cyclomatic_complexity_per_module', 'lines_of_comment', 'lines_of_comment_per_module', 'lines_of_code_per_line_of_comment', 'McCabes_cyclomatic_complexity_per_line_of_comment', 'IF4', 'IF4_per_module', 'IF4_visible', 'IF4_visible_per_module', 'IF4_concrete', 'IF4_concrete', 'rejected_lines_of_code\n'] 

['cnt_classes', 'max_member_funs', 'max_nested_loops', 'max_nesting_depth', 'max_params_in_decl', 'member_funs', 'member_vars']

Total label length before clearing: 37


In [20]:
def enlist_kept_cols(X, labels):
    cnt_cols = X.shape[1]
    X_reduced = []
    kept_cols = []
    
    for i in range(cnt_cols):
        col = X[:,i]
        notInside = True
        for j in range(len(X_reduced)):
            if (col == X_reduced[j]).all():
                notInside = False
        
        if notInside:
            kept_cols.append(i)
            X_reduced.append(col)

    return kept_cols

kept_cols = enlist_kept_cols(X, labels)
print (kept_cols, "\n")

X_reduced = X[:, kept_cols]
print (X_reduced.shape, "\n")

labels_reduced = [labels[i] for i in kept_cols]
print (labels_reduced)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 29, 30, 31, 32, 33, 34, 35, 36] 

(18667, 34) 

['number_of_modules', 'lines_of_code', 'lines_of_code_per_module', 'McCabes_cyclomatic_complexity', 'McCabes_cyclomatic_complexity_per_module', 'lines_of_comment', 'lines_of_comment_per_module', 'lines_of_code_per_line_of_comment', 'McCabes_cyclomatic_complexity_per_line_of_comment', 'IF4', 'IF4_per_module', 'IF4_visible', 'IF4_visible_per_module', 'IF4_concrete', 'IF4_concrete', 'rejected_lines_of_code\n', 'Files', 'Lines', 'AVG Len', 'Code', 'Comments', 'White SP', 'Cd/Cm+WS', 'Cd/Cm', 'Cd/WS', '% Code', 'cnt_classes', 'max_member_funs', 'max_nested_loops', 'max_nesting_depth', 'max_params_in_decl', 'member_funs', 'member_vars', 'min_member_funs\n']


In [21]:
import json
def create_json(author_names, dataset):
    data = {}
    data["column_descriptors"] = labels_reduced
    data["author_data"] = {}
    for author in author_names:
        feature_vecs, scores = get_author_features_and_scores(author, dataset)
        if len(scores) == 0:
            continue
        feat_vecs = [fv[kept_cols].tolist() for fv in feature_vecs]
        author_data = {}
        author_data["feature_vecs"] = feat_vecs
        author_data["scores"] = scores
        data["author_data"][author] = author_data
        
    with open('dataset.json', 'w') as outfile:
        json.dump(data, outfile)

#print get_author_features_and_scores(author_names[15], dataset)[1]
create_json(author_names, dataset)